In [ ]:
from keras.callbacks import TensorBoard
from keras.layers import Input, Convolution2D, MaxPooling2D, UpSampling2D, merge
from keras.models import Model
from keras import regularizers
import numpy as np
import keras.utils.vis_utils as vutil
from IPython.display import SVG, Image

In [ ]:
def inception_module(x, params, concat_axis,
                     subsample=(1, 1), activation='relu',
                     border_mode='same', weight_decay=None):

    # https://gist.github.com/nervanazoo/2e5be01095e935e90dd8  #
    # file-googlenet_neon-py

    (branch1, branch2, branch3, branch4) = params

    if weight_decay:
        W_regularizer = regularizers.l2(weight_decay)
        b_regularizer = regularizers.l2(weight_decay)
    else:
        W_regularizer = None
        b_regularizer = None

    pathway1 = Convolution2D(branch1[0], 1, 1,
                             subsample=subsample,
                             activation=activation,
                             border_mode=border_mode,
                             W_regularizer=W_regularizer,
                             b_regularizer=b_regularizer,
                             bias=False,)(x)

    pathway2 = Convolution2D(branch2[0], 1, 1,
                             subsample=subsample,
                             activation=activation,
                             border_mode=border_mode,
                             W_regularizer=W_regularizer,
                             b_regularizer=b_regularizer,
                             bias=False)(x)
    pathway2 = Convolution2D(branch2[1], 3, 1,
                             subsample=subsample,
                             activation=activation,
                             border_mode=border_mode,
                             W_regularizer=W_regularizer,
                             b_regularizer=b_regularizer,
                             bias=False)(pathway2)

    pathway3 = Convolution2D(branch3[0], 1, 1,
                             subsample=subsample,
                             activation=activation,
                             border_mode=border_mode,
                             W_regularizer=W_regularizer,
                             b_regularizer=b_regularizer,
                             bias=False)(x)
    pathway3 = Convolution2D(branch3[1], 5, 1,
                             subsample=subsample,
                             activation=activation,
                             border_mode=border_mode,
                             W_regularizer=W_regularizer,
                             b_regularizer=b_regularizer,
                             bias=False)(pathway3)

    pathway4 = MaxPooling2D(pool_size=(1, 1))(x)
    pathway4 = Convolution2D(branch4[0], 1, 1,
                             subsample=subsample,
                             activation=activation,
                             border_mode=border_mode,
                             W_regularizer=W_regularizer,
                             b_regularizer=b_regularizer,
                             bias=False)(pathway4)

    return merge([pathway1, pathway2, pathway3, pathway4],
                 mode='concat', concat_axis=concat_axis)

In [ ]:
from data_gen import get_data 

input_img = Input(shape=(1,540,1))

CONCAT_AXIS =1

#x = Convolution2D(16, 3, 1, activation='relu', border_mode='same')(input_img)
#x = MaxPooling2D((2,1), border_mode='same')(x)
x = inception_module(input_img, params=[(64, ), (1, 96 ), (1, 16 ), (32, )],
                     concat_axis=CONCAT_AXIS, border_mode='same')
#x = MaxPooling2D((2,1), border_mode='same')(x)
#x = Convolution2D(16, 3, 1, activation='relu', border_mode='same')(x)
x = inception_module(x, params=[(64, ), (1, 96 ), (1, 16 ), (32, )],
                     concat_axis=CONCAT_AXIS, border_mode='same')
#x = MaxPooling2D((2,1), border_mode='same')(x)
x = inception_module(x, params=[(64, ), (1, 96 ), (1, 16 ), (32, )],
                     concat_axis=CONCAT_AXIS, border_mode='same')

#encoded = MaxPooling2D((2,1), border_mode='same')(x)

#####

#x = Convolution2D(8, 3, 1, activation='relu', border_mode='same')(encoded)
#x = UpSampling2D((2,1))(x)
#x = Convolution2D(8, 3, 1, activation='relu', border_mode='same')(x)
#x = inception_module(encoded, params=[(64, ), (1, 96 ), (1, 16 ), (32, )],
#                     concat_axis=CONCAT_AXIS)
#x = UpSampling2D((2,1))(x)
#x = inception_module(x, params=[(64, ), (1, 96 ), (1, 16 ), (32, )],
#                     concat_axis=CONCAT_AXIS)
#x = UpSampling2D((2,1))(x)
#x = Convolution2D(8, 3, 1, activation='relu', border_mode='same')(x)
#x = UpSampling2D((2,1))(x)

decoded = Convolution2D(1, 3, 1, activation='relu', border_mode='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='rmsprop', loss='mse')

In [ ]:
Image(vutil.model_to_dot(autoencoder, show_shapes=True).create(prog='dot', format="jpeg"))

In [ ]:
epochs = 10;

for i in range(epochs):
    x, y, idx = get_data(20000)
    xtrain = x[:16000]
    ytrain = y[:16000]
    xval = x[16000:]
    yval = y[16000:]
    x_train = np.reshape(xtrain, (len(xtrain), 1, 540, 1))
    y_train = np.reshape(ytrain, (len(ytrain), 1, 540, 1))
    x_test = np.reshape(xval, (len(xval), 1, 540, 1))
    y_test = np.reshape(yval, (len(yval), 1, 540, 1))
    
    autoencoder.fit(x_train, y_train,
                     nb_epoch=10,
                     batch_size=32,
                     shuffle=False,
                     validation_data=(x_test, y_test),
                     callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

    #print("Epoch no = %d Training loss = %1.4f, Validation Loss = %1.4f" % (i,train_loss,test_loss))


In [ ]:
decoded_imgs = autoencoder.predict(x_test)

In [ ]:
%pylab inline

In [ ]:
# serialize model to JSON
model_json = autoencoder.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
autoencoder.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
plot(decoded_imgs[32].ravel())

In [ ]:
plot(y_test[32].ravel())

In [ ]:
plot(x_test[32].ravel())

In [ ]:
plot(x_test[342].ravel()[100:300])

In [ ]:
plot(y_test[342].ravel()[100:300])

In [ ]:
plot(decoded_imgs[342].ravel()[100:300])

In [ ]:
plot(y_test[32].ravel()) #142

In [ ]:
plot(decoded_imgs[32].ravel()) #142

In [ ]:
plot(x_test[32].ravel()) #142

In [ ]:
plot(x_test[342].ravel()) #142

In [ ]:
plot(decoded_imgs[342].ravel()) #142

In [ ]:
plot(y_test[342].ravel()) #142

In [ ]:
plot(decoded_imgs[439].ravel()) #142

In [ ]:
plot(x_test[439].ravel()) #142

In [ ]:
plot(y_test[439].ravel()) #142

In [ ]:
plot(decoded_imgs[90].ravel()) #142

In [ ]:
plot(x_test[90].ravel()) #142

In [ ]:
plot(decoded_imgs[707].ravel()) #142

In [ ]:
plot(x_train[707].ravel()) #142

In [ ]:
plot(y_train[707].ravel()) #142

In [ ]:
plot(y_train[990].ravel()) #142

In [ ]:
plot(x_train[990].ravel()) #142

In [ ]:
plot(decoded_imgs[990].ravel()) #142

In [ ]:
plot(decoded_imgs[22].ravel()) #142

In [ ]:
plot(x_test[22].ravel()) #142

In [ ]:
plot(y_test[22].ravel()) #142

In [ ]:
plot(y_test[1234].ravel()) #142

In [ ]:
plot(x_test[1234].ravel()) #60

In [ ]:
plot(decoded_imgs[1234].ravel()) #142

In [ ]:
plot(x_test[94].ravel()) #60

In [ ]:
plot(y_test[94].ravel()) #60

In [ ]:
plot(decoded_imgs[17].ravel())

In [ ]:
plot(x_test[17].ravel())

In [ ]:
plot(x_test[6].ravel())

In [ ]:
plot(decoded_imgs[6].ravel())

In [ ]:
plot(y_test[6].ravel())

In [ ]:
plot(y_test[73].ravel())

In [ ]:
plot(x_test[73].ravel())

In [ ]:
plot(decoded_imgs[73].ravel())

In [ ]:
plot(y_test[278].ravel())



In [ ]:
#plot(x_test[0])
plot(x_test[278].ravel())




In [ ]:
#plot(x_test[0])
plot(decoded_imgs[278].ravel())

In [ ]:
#plot(x_test[0])
plot(x_test[700].ravel())



In [ ]:
plot(y_test[700].ravel())

In [ ]:
plot(decoded_imgs[700].ravel())

In [ ]:
#plot(x_test[0])
plot(x_test[4].ravel())


In [ ]:
plot(y_test[4].ravel())

In [ ]:
plot(decoded_imgs[4].ravel())

In [ ]:
plot(decoded_imgs[10].ravel())

In [ ]:
print("Epoch no = %d Training loss = %1.4f, Validation Loss = %1.4f" % (i,train_loss,test_loss))